## Data Visualization

In [ ]:
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt
import os
import pandas as pd
import sqlite3

In [ ]:
class DatabaseHandler:
    def __init__(self, db_name):
        self.db_file = db_name
        self.conn = None

    def create_connection(self):
        try:
            self.conn = sqlite3.connect(self.db_file)
        except:
            print("Connection error")

    def create_table(self, create_table_query):
        try:
            c = self.conn.cursor()
            c.execute(create_table_query)
            # self.conn.commit()  # Uncomment this line if you want to commit changes immediately
        except:
            print("Create table error")

    def add_new_row(self, table,insert_list):
        # Insert new data
        try:
            cursor = self.conn.cursor()
            str_insert_list = "'" + "','".join(insert_list) + "'"
            add_new_row_query = f"""INSERT INTO {table}(board, id, pushes, author, title)
                                   VALUES({str_insert_list})"""

            cursor.execute(add_new_row_query)
            self.conn.commit()
        except:
            print('?')
            print(insert_list)
    
    def get_data(self,query):
        query_result = pd.read_sql(query,self.conn)
        return query_result

    def close_connection(self):
        if self.conn:
            self.conn.close()

In [ ]:
database_path = "Crawler\mydatabase1.db"
current_folder = os.getcwd()  # 取得目前程式碼所在的資料夾路徑
database_name = os.path.join(current_folder,database_path)  # 組合路徑
print(database_name)

# 確認目標database是否存在
if os.path.exists(database_name):
    print("database存在，可以進行讀取資料夾的操作。")
else:
    print("database不存在。")



handler = DatabaseHandler(database_name)
# create a database connection
handler.create_connection()

In [ ]:
query = """SELECT board,title FROM PTT_dealed"""
df = handler.get_data(query)

In [ ]:
TC_FONT_PATH = 'NotoSansTC-Black.otf' # 繁體中文字型檔名


plt.figure(figsize=(15,10))
k=1
for board_name in df['board'].unique():
    textlist =[]
    filter = df['board']==board_name
    for text in df[filter]['text']:
        for word in text:
            textlist.append(word)
    each_dictionary = Counter(textlist)
    wordcloud = WordCloud(background_color="white",contour_width=3, contour_color='black',font_path=TC_FONT_PATH).generate_from_frequencies(each_dictionary)
    
    plt.subplot(3,2,k)
    plt.imshow(wordcloud, interpolation="bilinear",aspect="auto")
    plt.axis("off")
    k+=1